In [11]:
# Load all required Libraries
import pandas as pd
import transformers, torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM

from sentence_transformers import SentenceTransformer

from datasets import Dataset

from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# Read Passages from the Datasets and Drop rows if they are NA or empty

In [7]:
passages = pd.read_parquet(
    "hf://datasets/rag-datasets/rag-mini-wikipedia/data/passages.parquet/part.0.parquet"
)

print(passages.shape)
passages.head()

(3200, 1)


,passage
id,
0,"Uruguay (official full name in ; pron. , Eas..."
1,"It is bordered by Brazil to the north, by Arge..."
2,Montevideo was founded by the Spanish in the e...
3,The economy is largely based in agriculture (m...
4,"According to Transparency International, Urugu..."


# Do EDA on the passage dataset
- You can try to find the maximum and minimum length of the passages before indexing (just a direction)

In [8]:
# Analyze passage lengths
passages["length"] = passages["passage"].str.len()
print(f"Min length: {passages['length'].min()}")
print(f"Max length: {passages['length'].max()}")
print(f"Mean length: {passages['length'].mean():.2f}")
print(f"Median length: {passages['length'].median()}")

# Check for missing values
print(f"\nMissing values: {passages['passage'].isna().sum()}")

Min length: 1
Max length: 2515
Mean length: 389.85
Median length: 299.0

Missing values: 0


# Tokenize Text and Generate Embeddings using Sentence Transformers

In [12]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode Text
embeddings = embedding_model.encode(
    passages["passage"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True,
    batch_size=64,
)

print(embeddings.shape)

d:\CMU\Applications of NLX and LLM\A2\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yash Thakkar\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed

torch.Size([3200, 384])


# Create Milvus Client and Insert your Embeddings to your DB
- Make sure you define a schema for your collection (Points will be deducted if you fail to define a proper schema with ids, passage text, embedding)

In [ ]:
# Define every column of your schema

id_ = ...
passage = ...
embedding = ...

In [ ]:
schema = ...

In [ ]:
client = MilvusClient("rag_wikipedia_mini.db")

# Create the Collection with Collection Name = "rag_mini". Make sure you define the schema variable while creating the collection

**Convert your Pandas Dataframe to a list of dictionaries**
- The Dictionary at least have 3 keys [id, passage, embedding]

In [ ]:
rag_data = ...

In [ ]:
# Code to insert the data to your DB
res = client.insert(collection_name="rag_mini", data=rag_data)

print(res)

- Do a Sanity Check on your database 

**Do not delete the below line during your submission**

In [ ]:
print("Entity count:", client.get_collection_stats("rag_mini")["row_count"])
print("Collection schema:", client.describe_collection("rag_mini"))

# Steps to Fetch Results
- Read the Question Dataset
- Clean the Question Dataset if necessary (Drop Questions with NaN etc.)
- Convert Each Query to a Vector Embedding (Use the same embedding model you used to embed your document)
- Try for a Single Question First
- Load Collection into Memory after creating Index for Search on your embedding field (This is an essential step before you can search in your db)
- Search and Fetch Top N Results

In [ ]:
import pandas as pd

queries = pd.read_parquet(
    "hf://datasets/rag-datasets/rag-mini-wikipedia/data/test.parquet/part.0.parquet"
)
queries

,question,answer
id,,
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
...,...,...
1710,Was Wilson president of the American Political...,Yes
1711,Did he not cast his ballot for John M. Palmer ...,Yes
1712,Did Wilson not spend 1914 through the beginnin...,Yes


In [ ]:
query = ...  # Your single query

query_embedding = ...

print(query_embedding.shape)

#### Create Index on the embedding column on your DB

In [ ]:
index_params = MilvusClient.prepare_index_params()

# Add an index on the embedding field
...

# Create the index
try:
    ...
except Exception as e:
    print(f"Index creation result: {e}")

# Load collection into memory (required for search)
...
print("Collection loaded into memory")

In [ ]:
# Search the db with your query embedding
output_ = ...

print(output_)

## Now get the Context 
- Initially use the first passage ONLY as your context
- In Later Experiments, you must try at least 2 different passage selection strategies (Top 3 / Top 5 / Top 10) and pass to your prompt

In [ ]:
context = ...

**Develop your Prompt**

In [ ]:
system_prompt = f""

prompt = f"""{system_prompt} \n Context: {context}: \n Question: {query} """
print(prompt)

# RAG Response for a Single Query

In [ ]:
# Load the LLM Model you want to use
...

In [ ]:
# Generate answer
...

# Decode and extract answer.
...

# Generate Responses for all the Queries in the Dataset

In [ ]:
# Your Code Here
...

# Finding out the Basic QA Metrics (F1 score, EM score)

In [ ]:
# Your code Here
...

# Advanced Evaluation using RAGAs

In [ ]:
data = {
    "question": ...,  # Question
    "answer": ...,  # Generated Answer
    "contexts": ...,  # Context you pass in. You can just use top-1 here
    "ground_truths": ...,  # Reference Answer in the dataset (Human annotated)
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
# Pass the dataset above to the evaluate method in RAGAs
# Your code here
...